In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [2]:
data = pd.read_csv('final_with_covid_AS.csv')
data.head()

,CountCases,BedDys,BeddystoCC,Severity,DiedCases,DiedBdDys,Hsptlsns,April,August,December,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,CovidPrd
0,17,133,7.823529,75,4,9,13,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,13,372,28.615385,63,4,34,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,106,13.250000,48,4,13,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,45,218,4.844444,153,3,24,42,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,24,238,9.916667,90,3,8,21,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
filtered_data2 = data[data['CarPngr'] == 1]
filtered_data2.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11789 entries, 33 to 54030
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CountCases       11789 non-null  int64  
 1   BedDys           11789 non-null  int64  
 2   BeddystoCC       11789 non-null  float64
 3   Severity         11789 non-null  int64  
 4   DiedCases        11789 non-null  int64  
 5   DiedBdDys        11789 non-null  int64  
 6   Hsptlsns         11789 non-null  int64  
 7   April            11789 non-null  int64  
 8   August           11789 non-null  int64  
 9   December         11789 non-null  int64  
 10  February         11789 non-null  int64  
 11  January          11789 non-null  int64  
 12  July             11789 non-null  int64  
 13  June             11789 non-null  int64  
 14  March            11789 non-null  int64  
 15  May              11789 non-null  int64  
 16  November         11789 non-null  int64  
 17  October         

In [5]:
num_bins = 2

bed_days_min = filtered_data2['Severity'].min()
bed_days_max = filtered_data2['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data2['Bed_days_category'] = filtered_data2['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_1341/530833198.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data2['Bed_days_category'] = filtered_data2['Severity'].apply(bin_function)


In [6]:
filtered_data2['Bed_days_category'].value_counts()


Bed_days_category
0    11589
1      200
Name: count, dtype: int64

In [7]:

from xgboost import XGBClassifier

X = filtered_data2[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   
                   'Male','Female',
                   
                   'Regional','MajorCities',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]



y = filtered_data2['Bed_days_category']


Xgboost

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before calibration
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(xgb_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predictions and probabilities after calibration
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    confusion_matrices_after.append(confusion_matrix(y_test, y_pred_after))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Calibration:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_after)} After Calibration:\n', confusion_matrices_after[-1])

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix before and after calibration
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0).astype(int)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0).astype(int)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Confusion Matrix for Fold 1 Before Calibration:
 [[1154    5]
 [   9   11]]
Confusion Matrix for Fold 1 After Calibration:
 [[1141   18]
 [   1   19]]
Confusion Matrix for Fold 2 Before Calibration:
 [[1150    9]
 [   5   15]]
Confusion Matrix for Fold 2 After Calibration:
 [[1134   25]
 [   1   19]]
Confusion Matrix for Fold 3 Before Calibration:
 [[1151    8]
 [  11    9]]
Confusion Matrix for Fold 3 After Calibration:
 [[1135   24]
 [   2   18]]
Confusion Matrix for Fold 4 Before Calibration:
 [[1154    5]
 [   6   14]]
Confusion Matrix for Fold 4 After Calibration:
 [[1143   16]
 [   1   19]]
Confusion Matrix for Fold 5 Before Calibration:
 [[1154    5]
 [   8   12]]
Confusion Matrix for Fold 5 After Calibration:
 [[1141   18]
 [   1   19]]
Confusion Matrix for Fold 6 Before Calibration:
 [[1155    4]
 [   5   15]]
Confusion Matrix for Fold 6 After Calibration:
 [[1140   19]
 [   2   18]]
Confusion Matrix for Fold 7 Before Calibration:
 [[1154    5]
 [   6   14]]
Confusion Matrix f

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    lr_model = LogisticRegression( random_state=42)
    lr_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_lr_before = lr_model.predict(X_test)
    y_prob_lr_before = lr_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_lr_before))
    precision_list_before.append(precision_score(y_test, y_pred_lr_before))
    recall_list_before.append(recall_score(y_test, y_pred_lr_before))
    f1_list_before.append(f1_score(y_test, y_pred_lr_before))
    entropy_list_before.append(calculate_entropy(y_prob_lr_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_lr_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(lr_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_lr_after = calibrated_model.predict(X_test)
    y_prob_lr_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_lr_after))
    precision_list_after.append(precision_score(y_test, y_pred_lr_after))
    recall_list_after.append(recall_score(y_test, y_pred_lr_after))
    f1_list_after.append(f1_score(y_test, y_pred_lr_after))
    entropy_list_after.append(calculate_entropy(y_prob_lr_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_lr_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


--- Before Calibration ---
Mean Accuracy: 0.9895662780031422
Accuracy Std Dev: 0.0022472782060769785
Mean Precision: 0.7786629469601606
Precision Std Dev: 0.08904871702007784
Mean Recall: 0.5399999999999999
Recall Std Dev: 0.12409673645990857
Mean F1-score: 0.6303604848576385
F1-score Std Dev: 0.10258351999171707
Mean Entropy: 0.03229952351742282
Entropy Std Dev: 0.0021293695552566253
Mean Confusion Matrix:
 [[1155.8    3.1]
 [   9.2   10.8]]
--- After Calibration ---
Mean Accuracy: 0.9830349595568171
Accuracy Std Dev: 0.0031052518312699655
Mean Precision: 0.5045579657865285
Precision Std Dev: 0.04803729833547105
Mean Recall: 0.93
Recall Std Dev: 0.04582575694955839
Mean F1-score: 0.6525605058958674
F1-score Std Dev: 0.04287528122988752
Mean Entropy: 0.02378723625442321
Entropy Std Dev: 0.0022117323843486515
Mean Confusion Matrix:
 [[1140.3   18.6]
 [   1.4   18.6]]


decesion Tree

In [11]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_dt_before = dt_model.predict(X_test)
    y_prob_dt_before = dt_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_dt_before))
    precision_list_before.append(precision_score(y_test, y_pred_dt_before))
    recall_list_before.append(recall_score(y_test, y_pred_dt_before))
    f1_list_before.append(f1_score(y_test, y_pred_dt_before))
    entropy_list_before.append(calculate_entropy(y_prob_dt_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_dt_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(dt_model, method='sigmoid', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_dt_after = calibrated_model.predict(X_test)
    y_prob_dt_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_dt_after))
    precision_list_after.append(precision_score(y_test, y_pred_dt_after))
    recall_list_after.append(recall_score(y_test, y_pred_dt_after))
    f1_list_after.append(f1_score(y_test, y_pred_dt_after))
    entropy_list_after.append(calculate_entropy(y_prob_dt_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_dt_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


--- Before Calibration ---
Mean Accuracy: 0.9860886826768965
Accuracy Std Dev: 0.0026877626096095835
Mean Precision: 0.5960508501646272
Precision Std Dev: 0.09221564298022161
Mean Recall: 0.5549999999999999
Recall Std Dev: 0.0986154146165801
Mean F1-score: 0.573306520912426
F1-score Std Dev: 0.09238552409938026
Mean Entropy: 2.4025851012630826e-09
Entropy Std Dev: 0.0
Mean Confusion Matrix:
 [[1151.4    7.5]
 [   8.9   11.1]]
--- After Calibration ---
Mean Accuracy: 0.9868518974527347
Accuracy Std Dev: 0.00271698773543693
Mean Precision: 0.6366125261906996
Precision Std Dev: 0.09820803661380632
Mean Recall: 0.505
Recall Std Dev: 0.12338962679253065
Mean F1-score: 0.5602072166758074
F1-score Std Dev: 0.1133595843291618
Mean Entropy: 0.05841113188918666
Entropy Std Dev: 0.0020880545526119174
Mean Confusion Matrix:
 [[1153.3    5.6]
 [   9.9   10.1]]


Svm

In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the base SVM model
    svm_model = SVC(probability=True, random_state=42)
    svm_model.fit(X_train, y_train)
    
    # 2. Predict before calibration
    y_pred_before = svm_model.predict(X_test)
    y_prob_before = svm_model.predict_proba(X_test)
    
    # 3. Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # 4. Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_before)
    confusion_matrices_before.append(conf_matrix_before)
    
    # 5. Calibrate the model
    calibrated_model = CalibratedClassifierCV(svm_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # 6. Predict with the calibrated model
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # 7. Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # 8. Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation for each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)

# Calculate mean and standard deviation for each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results
print('Before Calibration:')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

print('\nAfter Calibration:')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Before Calibration:
Mean Accuracy: 0.9894814603610727
Accuracy Std Dev: 0.0018678754989892
Mean Precision: 0.7938468394350748
Precision Std Dev: 0.07788551151000622
Mean Recall: 0.5149999999999999
Recall Std Dev: 0.10259142264341595
Mean F1-score: 0.6195569988715149
F1-score Std Dev: 0.08508930800216104
Mean Entropy: 0.03695803280953094
Entropy Std Dev: 0.001506418885996128
Mean Confusion Matrix:
 [[1156.2    2.7]
 [   9.7   10.3]]

After Calibration:
Mean Accuracy: 0.9885487543038833
Accuracy Std Dev: 0.001574722001824661
Mean Precision: 0.6476002068107333
Precision Std Dev: 0.04580885408718834
Mean Recall: 0.7200000000000001
Recall Std Dev: 0.0678232998312527
Mean F1-score: 0.6802520508830158
F1-score Std Dev: 0.0452193174373126
Mean Entropy: 0.040920907315643115
Entropy Std Dev: 0.0029117176937377884
Mean Confusion Matrix:
 [[1151.     7.9]
 [   5.6   14.4]]


In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the base Random Forest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    
    # 2. Predict before calibration
    y_pred_before = rf_model.predict(X_test)
    y_prob_before = rf_model.predict_proba(X_test)
    
    # 3. Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # 4. Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_before)
    confusion_matrices_before.append(conf_matrix_before)
    
    # 5. Calibrate the model
    calibrated_model = CalibratedClassifierCV(rf_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # 6. Predict with the calibrated model
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # 7. Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # 8. Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation for each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)

# Calculate mean and standard deviation for each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results
print('Before Calibration:')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

print('\nAfter Calibration:')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Before Calibration:
Mean Accuracy: 0.9882090517272415
Accuracy Std Dev: 0.0027487395340848202
Mean Precision: 0.7075923831070889
Precision Std Dev: 0.09856100992516917
Mean Recall: 0.51
Recall Std Dev: 0.15132745950421556
Mean F1-score: 0.5832725457725457
F1-score Std Dev: 0.13632524037064597
Mean Entropy: 0.024913575102664336
Entropy Std Dev: 0.0023094719348853584
Mean Confusion Matrix:
 [[1154.8    4.1]
 [   9.8   10.2]]

After Calibration:
Mean Accuracy: 0.984901307689317
Accuracy Std Dev: 0.0035739468200470855
Mean Precision: 0.5410757517169017
Precision Std Dev: 0.06780904413090155
Mean Recall: 0.8099999999999999
Recall Std Dev: 0.09433981132056601
Mean F1-score: 0.646652695435593
F1-score Std Dev: 0.07107346817808528
Mean Entropy: 0.030329303092535925
Entropy Std Dev: 0.0032616989083908395
Mean Confusion Matrix:
 [[1144.9   14. ]
 [   3.8   16.2]]
